In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
import string
import unicodedata
import re

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def clean_text(text):
    return unicodeToAscii(re.sub(r"”|“",'"',text).replace("’","'"))


def build_structure(input):
    acc = []
    stack = []
    for token in input:
        if token == "'s":
            acc = acc[:-1] + [(acc[-1] + token) if len(acc) > 0 else token]
        elif token in (list(string.punctuation) + ["''","``","’"]):
            stack += [(' '.join(acc),token + ' ')]
            acc = []
        else:
            acc += [token]
    stack += [(' '.join(acc),None)]
    res = None
    for x in stack[::-1]:
        res = (x[0],x[1],res)
    return res


def structure_to_sentences(structure):
    res = []
    next = structure
    while next is not None:
        str, sep, next = next
        if len(str) > 0:
            res += [str]
    return res

def transf(s):
    return structure_to_sentences(build_structure(word_tokenize(clean_text(s))))

In [41]:
transf('ns over Europe\'s debt heat up again Barnes and Noble considers Nook spin-off')

ns over Europe's debt heat up again Barnes and Noble considers Nook spin-off


["ns over Europe's debt heat up again Barnes and Noble considers Nook spin-off"]

In [43]:
for df_index in range(1,6):
    print(f'index: {df_index}')
    df = pd.read_json(f'./train{df_index}.json',lines=True)
    f = open(f'./dataset{df_index}.txt','w')
    for index, row in df.iterrows():
        sentences = transf(row['text'])
        for sentence in sentences:
            f.write(sentence+'\n')
    f.close()

index: 1
index: 2
index: 3
index: 4
index: 5
